# Yukon Chinook Environmental Data Extraction for Forecast
## Purpose
The Yukon River Delta Chinook Salmon run timing and outlook depend on the environmental conditions in the Bering Sea each year. The purpose of this notebook is to extract that data, do some minor processing and/or derivations, and save the time-series of the outputs as CSV files.

## Inputs
The timing model is dependent on the following environmental conditions:

### Mean Sea Ice Concentration, March 20 - June 1
- "Average Sea Ice Concentration (%) Shpanberg Strait (62–63°N, 166–169°W)"
- Extracted from the NSIDC Near-Real-Time DMSP SSMIS Daily Polar Gridded Sea Ice Concentrations, Version 1
- Original: https://nsidc.org/data/NSIDC-0081/versions/1
- AOOS: https://portal.aoos.org/#module-metadata/391183ee-827e-11e1-a4f3-00219bfe5678/c4d14166-cae8-4bb0-8cd5-fc876f07d63c
- RW Directory: `/data/packrat/nsids/nsidc0081/processed/2023/`

### Mean Marine Surface Temperature, May 1 - May 31

- "Marine Surface Temperature (°C) 26.5 mi due west of Nunaktuk Island (Middle Mouth) (63.1°N 165.5°W)"- unlike Sea Ice, this is just extracted from a single location.
- Extracted from NCEP/NCAR Reanalysis 1: Surface Flux dataset
- Original: https://psl.noaa.gov/data/gridded/data.ncep.reanalysis.surfaceflux.html
- AOOS: https://portal.aoos.org/#module-metadata/071705de-8400-11e1-99fe-00219bfe5678/047e7934-2ed7-431f-8a85-0bdd8c6a08cd
- RW Directory: `/data/packrat/ncep/reanalysis/dailyavgs/surface_gauss/processed/2023/2023_01/surface_gauss_2023.nc`

### Air Temperature in Nome, Alaska, April 1 - April 30

- "Air Temperature (°C) for Nome, AK"
- Recorded by National Weather Service station PAOM
- Original: https://w1.weather.gov/data/obhistory/PAOM.html
- AOOS: https://portal.aoos.org/#metadata/14059/station
- AOOS ERDDAP: http://erddap.aoos.org/erddap/tabledap/gov_noaa_nws_paom.html
- Note: there are multiple weather stations in Nome. Other options include a [Marine Exchange of Alaska weather station](https://portal.aoos.org/#metadata/103371/station) and a [CO-OPS station](https://portal.aoos.org/#metadata/12017/station).

## Outputs
This notebook can be used to get the following:
- CSV extractions of the environmental data for the given periods.

## Modification History
- 2021.05.13: Protyping WIP (W. Koeppen, Axiom)
- 2023.05.05: Updating file paths (Aidan Lewis, Axiom)

In [1]:
import glob
import os
import sys
import pandas as pd
from datetime import datetime
import numpy as np
import netCDF4
import xarray as xr
from tqdm import trange
import matplotlib.pyplot as plt
import warnings

## Set up models

Setting the year will propogate through all three requests.

In [2]:
year = 2023

Some configurations. I didn't use all of these, but it's kind of useful to collect these here anyway.

In [3]:
data_config = {
    "sea_ice": {
        "title": "NSIDC Sea Ice",
        "path": "/data/packrat/nsidc/nsidc0081/processed/2023/",
        "data_varname": "Sea_Ice_Concentration",
        "lon_varname": "longitude",
        "lat_varname": "latitude",
        "time_varname": "time",
    },
     "surface_temp": {
         "title": "NCEP Surface Temperature",
         "path": "/data/packrat/ncep/reanalysis/dailyavgs/surface_gauss/processed/2023/2023_01/",
         "data_varname": "skt",
         "lon_varname": "lon",
         "lat_varname": "lat",
         "time_varname": "time",
     }
}

This indexes the files on disk, listing the file location, time available, and integer index of each time. You don't have to do this every time you want a new year. But you'll have to do it at least once.

In [4]:
def time_file_index(search_string, time_var):
    
    filenames = sorted(glob.glob(search_string))
    nfiles = len(filenames)
    
    # Build a tuple with dates and filenames the file contains for every file in the index
    time_file = []

    for i in trange(nfiles):

        with netCDF4.Dataset(filenames[i]) as netcdf:
            # extract the time, turn it into a date
            
            time_dat = netcdf.variables[time_var]
            times = np.array(time_dat)
            
            # some have calendar, some don't
            try:
                times = netCDF4.num2date(times, time_dat.units, time_dat.calendar, only_use_python_datetimes=True)
            except:
                times = netCDF4.num2date(times, time_dat.units)
        
        for j in range(len(times)):
            time_file.append((times[j], filenames[i], j))
    
    
    result = pd.DataFrame(time_file, columns=['date', 'file', 't_index'])
    time_format="%Y-%m-%d %H:%M:%S"
    result['date'] = [pd.Timestamp(t.strftime(time_format)) for t in result['date']]
    result = result.set_index('date')

    #check for duplicates
    dupes = result.index.duplicated(keep='first')
    
    if dupes.sum() > 0:
 #       print('Found duplicate times, using first one found.')
        result = result[~dupes]
    
    return result

## NSIDC Sea Ice
A few notes here:

- This method select only the points that have latitude and longitudes within our bounds. This dataset provides the centroids of the cells, so this method won't take into account partial pixels. A cell is either in or out.
- The data is projected, so we can't select centroids by their integer location. Instead we select them by their latitude and longitude labels.
- Xarray throws a warning when opening these files: `SerializationWarning: variable 'Sea_Ice_Concentration' has multiple fill values {-9999, 255}, decoding all values to NaN.` It's not a problem for this work, so I supress these with a warnings catch.

In [5]:
ice_time_file = time_file_index(os.path.join(data_config["sea_ice"]["path"], '**/*.nc'), data_config["sea_ice"]["time_varname"])

100%|██████████| 153/153 [00:01<00:00, 81.68it/s]


In [6]:
ice_file_list = ice_time_file.loc[f"{year}-03-20":f"{year}-06-01"]

In [7]:
def get_ice_values(row):
    
    """Get the mean Sea Ice Concentration value in a lat/lon box."""
    
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=xr.SerializationWarning)
        dat = xr.open_dataset(row['file']).isel(time=row['t_index'])
    
    masked = dat['Sea_Ice_Concentration'].\
        where(dat['latitude'] >= 62).\
        where(dat['latitude'] <= 63).\
        where(dat['longitude'] >= -169).\
        where(dat['longitude'] <= -166)
    subset = masked.values[~np.isnan(masked.values)]
    result = np.nanmean(subset)
    
    return result

vals = []
for i in trange(len(ice_file_list)):
    vals.append(get_ice_values(ice_file_list.iloc[i]))

ice_df = pd.DataFrame({"mean_sea_ice_percent":vals}, index=ice_file_list.index)

100%|██████████| 73/73 [00:04<00:00, 16.16it/s]


In [8]:
ice_df.to_csv(f"../outputs/sea_ice_concentration-{year}.csv")

## NCEP Skin Temperature
Notes:
- longitudes here are from 0 to 360 positive east. So 63.1°N 165.5°W becomes 63.1°N 194.5°E
- there are no exact matches here, but we get pretty close (centroid is at 63.8079°N 195.0°E).

In [9]:
skt_time_file = time_file_index(os.path.join(data_config["surface_temp"]["path"], '*.nc'), data_config["surface_temp"]["time_varname"])

100%|██████████| 1/1 [00:00<00:00,  1.38it/s]


In [10]:
skt_file_list = skt_time_file.loc[f"{year}-05-01":f"{year}-05-31"]

In [11]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx


def get_skt_values(row):
    
    """Get the skin temperature value at the lat/lon."""
    
    dat = xr.open_dataset(row['file']).isel(time=row['t_index'])
    
    lat_index = find_nearest(dat['lat'], 63.1)
    lon_index = find_nearest(dat['lon'], 194.5)
    
    result = dat["skt"].isel(lat=lat_index, lon=lon_index).item()
    
    return result

vals = []
for i in trange(len(skt_file_list)):
    vals.append(get_skt_values(skt_file_list.iloc[i]))

skt_df = pd.DataFrame({"surface_temp_c":vals}, index=skt_file_list.index)

100%|██████████| 31/31 [00:01<00:00, 18.23it/s]


In [12]:
skt_df.to_csv(f"../outputs/surface_temperature-{year}.csv")

## Nome Air Temperature
Let's just get this from [AOOS ERDDAP](http://erddap.aoos.org/erddap/index.html).
- This station provides hourly data in Celsius.
- We can re-sample to daily to match the other datasets. But I'll add another column that is the count of the number of values in each daily mean, so we know if there are missing data.
- Data in our archive begins on May 16, 2015, so this will fail if years before that are selected.

In [13]:
def erddap_sensor(
    stationurl,
    time_min=datetime(2020,9,1,0,0),
    time_max=datetime(2020,12,31,0,0)
):
    
    url = f"{stationurl}.csv" \
    f"?time,latitude,longitude,air_temperature" \
    f"&time>={time_min.strftime('%Y-%m-%dT%H:%M:%SZ')}" \
    f"&time<={time_max.strftime('%Y-%m-%dT%H:%M:%SZ')}"
            
    print(url)
    result = pd.read_csv(url, parse_dates=True, index_col='time', skiprows=[1])
    result = result.sort_index() # put it in order
    
    return result

In [14]:
request_data = erddap_sensor(
    "http://erddap.aoos.org/erddap/tabledap/gov_noaa_awc_paom",
    time_min=datetime(year,4,1,0,0),
    time_max=datetime(year,5,1,0,0)
)

http://erddap.aoos.org/erddap/tabledap/gov_noaa_awc_paom.csv?time,latitude,longitude,air_temperature&time>=2023-04-01T00:00:00Z&time<=2023-05-01T00:00:00Z


In [15]:
resample = request_data["air_temperature"].resample('1D')

nome_df = pd.DataFrame({
    "air_temperature_c": resample.mean(),
    "n_observations": resample.count()
    
}, index=resample.mean().index)
nome_df.index.name = "date"

In [16]:
nome_df.to_csv(f"../outputs/nome_air_temp-{year}.csv")